In [13]:
from google.colab import drive
drive.mount('/content/drive')
import kagglehub
import os
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torchvision.transforms.functional import InterpolationMode
import torch.backends.cudnn as cudnn
from torch.cuda.amp import autocast, GradScaler

nikhilroxtomar_person_segmentation_path = kagglehub.dataset_download('nikhilroxtomar/person-segmentation')
tapakah68_supervisely_filtered_segmentation_person_dataset_path = kagglehub.dataset_download('tapakah68/supervisely-filtered-segmentation-person-dataset')

print('Data source import complete.')
print(nikhilroxtomar_person_segmentation_path)
print(tapakah68_supervisely_filtered_segmentation_person_dataset_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using Colab cache for faster access to the 'person-segmentation' dataset.
Using Colab cache for faster access to the 'supervisely-filtered-segmentation-person-dataset' dataset.
Data source import complete.
/kaggle/input/person-segmentation
/kaggle/input/supervisely-filtered-segmentation-person-dataset


In [14]:
images_path =  "/kaggle/input/supervisely-filtered-segmentation-person-dataset/supervisely_person_clean_2667_img/supervisely_person_clean_2667_img/"
masks_path  = "/kaggle/input/supervisely-filtered-segmentation-person-dataset/supervisely_person_clean_2667_img/supervisely_person_clean_2667_img/"

images_path2="/kaggle/input/person-segmentation/people_segmentation/images/"
masks_path2="/kaggle/input/person-segmentation/people_segmentation/masks/"
df =  pd.read_csv('/kaggle/input/supervisely-filtered-segmentation-person-dataset/df.csv')
df.head()
images_path2_list = sorted(os.listdir(images_path2))
masks_path2_list = sorted(os.listdir(masks_path2))
df2 = df[["images", "masks"]].copy()
df2["images"] = df2['images'].apply(lambda x: images_path + x)
df2["masks"]  = df2['masks'].apply(lambda x: masks_path + x)
df2["coef"]   = 1

df3 = pd.DataFrame({
    "images": [images_path2 + elt for elt in images_path2_list],
    "masks":  [masks_path2 + elt for elt in masks_path2_list],
    "coef":   255
})

final_df = pd.concat([df2, df3], ignore_index=True)

X_train_raw, X_test_raw  =  train_test_split(final_df, test_size=0.1, random_state=42)

In [15]:
final_df = final_df.copy()
final_df["source"] = np.where(final_df["coef"] == 1, "supervisely", "person_seg")
print("Tổng số mẫu:", len(final_df))
print("\nSố mẫu theo nguồn:")
print(final_df["source"].value_counts())

def dice_loss(logits, target, eps=1e-6):
    pred = torch.sigmoid(logits)
    target = target.float()

    pred = pred.view(pred.size(0), -1)
    target = target.view(target.size(0), -1)

    inter = (pred * target).sum(dim=1)
    denom = pred.sum(dim=1) + target.sum(dim=1)
    dice = (2 * inter + eps) / (denom + eps)
    return 1 - dice.mean()

def dice_score_from_logits(logits, target, thr=0.5, eps=1e-6):
    prob = torch.sigmoid(logits)
    pred = (prob > thr).float()
    inter = (pred * target).sum()
    union = pred.sum() + target.sum()
    return ((2*inter + eps) / (union + eps)).item()

def logits_to_probs_preds(logits, thr=0.5):
    probs = torch.sigmoid(logits)
    preds = (probs > thr).float()
    return probs, preds
def update_global_stats_from_logits(logits, targets, thr=0.5):
    """
    logits: (B,1,H,W) raw logits
    targets: (B,1,H,W) {0,1}
    Return: tp, fp, fn, correct, total (all as python floats/ints)
    """
    probs = torch.sigmoid(logits)
    preds = (probs > thr).float()

    tp = (preds * targets).sum().item()
    fp = (preds * (1 - targets)).sum().item()
    fn = ((1 - preds) * targets).sum().item()

    correct = (preds == targets).float().sum().item()
    total = targets.numel()

    return tp, fp, fn, correct, total

Tổng số mẫu: 8345

Số mẫu theo nguồn:
source
person_seg     5678
supervisely    2667
Name: count, dtype: int64


In [17]:
SEED = 42

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]
import random, os
import numpy as np
import torch

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def seed_worker(worker_id):
    # đảm bảo mỗi worker có seed khác nhau nhưng reproducible
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


from torch.utils.data import Dataset
from PIL import Image, ImageOps
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torchvision.transforms import InterpolationMode

# ====== GIỐNG RAW: thay mean/std theo RAW của bạn ======
RAW_MEAN = [0.485, 0.456, 0.406]  # <-- nếu raw khác, thay số ở đây
RAW_STD  = [0.229, 0.224, 0.225]  # <-- nếu raw khác, thay số ở đây

class PreprocessedSegDataset(Dataset):
    def __init__(self, df, size=512, augment=False):
        self.df = df.reset_index(drop=True)
        self.size = size
        self.augment = augment

        # tạo mean/std 1 lần (nhanh hơn)
        self.mean = torch.tensor(RAW_MEAN, dtype=torch.float32).view(3,1,1)
        self.std  = torch.tensor(RAW_STD,  dtype=torch.float32).view(3,1,1)

        # augment giống kiểu bạn đang dùng
        self.color_jitter = T.ColorJitter(
            brightness=0.15, contrast=0.15, saturation=0.1, hue=0.02
        )

    def __len__(self):
        return len(self.df)

    def _resize(self, img, mask):
        img  = img.resize((self.size, self.size), Image.BILINEAR)
        mask = mask.resize((self.size, self.size), Image.NEAREST)
        return img, mask

    def _augment(self, img, mask):
        # Horizontal flip
        if random.random() < 0.5:
            img = ImageOps.mirror(img)
            mask = ImageOps.mirror(mask)

        # small rotation
        if random.random() < 0.3:
            angle = random.uniform(-12, 12)
            img  = TF.rotate(img, angle, interpolation=InterpolationMode.BILINEAR, expand=False)
            mask = TF.rotate(mask, angle, interpolation=InterpolationMode.NEAREST, expand=False)

        # random crop + resize (mild)
        if random.random() < 0.25:
            w, h = img.size
            scale = random.uniform(0.88, 1.0)
            new_w, new_h = int(w*scale), int(h*scale)
            left = random.randint(0, max(0, w-new_w))
            top  = random.randint(0, max(0, h-new_h))
            img  = img.crop((left, top, left+new_w, top+new_h))
            mask = mask.crop((left, top, left+new_w, top+new_h))
            img  = img.resize((self.size, self.size), Image.BILINEAR)
            mask = mask.resize((self.size, self.size), Image.NEAREST)

        # color jitter chỉ áp cho ảnh
        img = self.color_jitter(img)
        return img, mask

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_path  = row["images"]
        mask_path = row["masks"]

        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        # resize trước (để batch shape cố định)
        img, mask = self._resize(img, mask)

        # augment
        if self.augment:
            img, mask = self._augment(img, mask)

        # to numpy
        img_np  = np.array(img, dtype=np.float32) / 255.0
        mask_np = np.array(mask, dtype=np.float32)

        # ====== GIỐNG RAW: mask -> {0,1} robust ======
        max_val = mask_np.max() if mask_np.max() > 0 else 1.0
        if max_val > 1.0:
            mask_np = mask_np / max_val
        mask_np = (mask_np >= 0.5).astype(np.float32)

        # to tensor
        img_t  = torch.from_numpy(img_np).permute(2,0,1).float()
        mask_t = torch.from_numpy(mask_np).unsqueeze(0).float()

        # ====== GIỐNG RAW: normalize theo RAW mean/std ======
        img_t = (img_t - self.mean) / self.std

        return img_t, mask_t


class DoubleConv(nn.Module):
    """(Conv => BN => ReLU) * 2"""
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.net(x)

class Down(nn.Module):
    """Downscaling with maxpool then double conv"""
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_ch, out_ch)
        )

    def forward(self, x):
        return self.net(x)

class Up(nn.Module):
    """Upscaling then double conv (skip connections)"""
    def __init__(self, in_ch, out_ch, bilinear=True):
        super().__init__()
        self.bilinear = bilinear

        if bilinear:
            # in_ch is concat channels (from skip + upsampled), we'll reduce inside DoubleConv
            self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False)
            self.conv = DoubleConv(in_ch, out_ch)
        else:
            # transposed conv halves channels
            self.up = nn.ConvTranspose2d(in_ch // 2, in_ch // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x1, x2):
        # x1: from decoder (lower res), x2: from encoder skip (higher res)
        x1 = self.up(x1)

        # Pad if needed (in case of odd sizes)
        diffY = x2.size(2) - x1.size(2)
        diffX = x2.size(3) - x1.size(3)
        if diffY != 0 or diffX != 0:
            x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                            diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=1, base_c=64, bilinear=True):
        super().__init__()
        self.in_conv = DoubleConv(in_channels, base_c)
        self.down1 = Down(base_c, base_c * 2)      # 64 -> 128
        self.down2 = Down(base_c * 2, base_c * 4)  # 128 -> 256
        self.down3 = Down(base_c * 4, base_c * 8)  # 256 -> 512

        factor = 2 if bilinear else 1
        self.down4 = Down(base_c * 8, base_c * 16 // factor)  # 512 -> 1024 (or 512 if bilinear)

        self.up1 = Up(base_c * 16, base_c * 8 // factor, bilinear=bilinear)
        self.up2 = Up(base_c * 8, base_c * 4 // factor, bilinear=bilinear)
        self.up3 = Up(base_c * 4, base_c * 2 // factor, bilinear=bilinear)
        self.up4 = Up(base_c * 2, base_c, bilinear=bilinear)

        self.out_conv = nn.Conv2d(base_c, num_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.in_conv(x)   # 512
        x2 = self.down1(x1)    # 256
        x3 = self.down2(x2)    # 128
        x4 = self.down3(x3)    # 64
        x5 = self.down4(x4)    # 32

        x = self.up1(x5, x4)   # 64
        x = self.up2(x,  x3)   # 128
        x = self.up3(x,  x2)   # 256
        x = self.up4(x,  x1)   # 512

        logits = self.out_conv(x)  # (B,1,H,W)
        return logits

history_name = "training_unet_history_preprocessed_metrics.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(in_channels=3, num_classes=1, base_c=32, bilinear=True).to(device)

criterion_bce = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

from torch.utils.data import DataLoader

train_ds = PreprocessedSegDataset(X_train_raw, size=256, augment=True)
val_ds   = PreprocessedSegDataset(X_test_raw,  size=256, augment=False)

g = torch.Generator()
g.manual_seed(SEED)

train_loader = DataLoader(
    train_ds,
    batch_size=8,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    worker_init_fn=seed_worker,
    generator=g
)

val_loader = DataLoader(
    val_ds,
    batch_size=8,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    worker_init_fn=seed_worker,
    generator=g
)


num_epochs = 20
history = {
    "train_loss": [],
    "val_loss": [],
    "val_iou": [],
    "val_dice": [],
    "val_acc": [],
    "val_prec": [],
    "val_recall": [],
    "val_f1": []
}

for epoch in range(num_epochs):
    # ------------------ TRAIN ------------------
    model.train()
    running_loss = 0.0

    # global accumulators
    tp = fp = fn = 0.0
    correct = 0.0
    total = 0

    for imgs, masks in tqdm(train_loader, desc=f"Train epoch {epoch+1}"):
        imgs = imgs.to(device).float()
        masks = masks.to(device).float()

        logits = model(imgs)

        bce = criterion_bce(logits, masks)
        dsc = dice_loss(logits, masks)
        loss = bce + dsc

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        with torch.no_grad():
            _tp, _fp, _fn, _correct, _total = update_global_stats_from_logits(logits, masks, thr=0.5)
            tp += _tp
            fp += _fp
            fn += _fn
            correct += _correct
            total += _total

    avg_loss = running_loss / max(1, len(train_loader))

    # global epoch metrics
    train_iou  = tp / (tp + fp + fn + 1e-6)
    train_dice = (2*tp) / (2*tp + fp + fn + 1e-6)
    train_acc  = correct / max(1, total)

    print(f"Epoch {epoch+1} - train avg loss: {avg_loss:.4f} | IoU: {train_iou:.4f} | Dice: {train_dice:.4f} | Acc: {train_acc:.4f}")
    history["train_loss"].append(avg_loss)

    # ------------------ VAL ------------------
    model.eval()
    val_loss = 0.0
    val_steps = 0

    # global accumulators for val
    v_tp = v_fp = v_fn = 0.0
    v_correct = 0.0
    v_total = 0

    # global accumulators for precision/recall/f1
    # (they are derived from tp/fp/fn, so no need separate sums)
    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs = imgs.to(device).float()
            masks = masks.to(device).float()

            logits = model(imgs)

            bce = criterion_bce(logits, masks)
            dsc = dice_loss(logits, masks)
            batch_loss = (bce + dsc).item()
            val_loss += batch_loss
            val_steps += 1

            _tp, _fp, _fn, _correct, _total = update_global_stats_from_logits(logits, masks, thr=0.5)
            v_tp += _tp
            v_fp += _fp
            v_fn += _fn
            v_correct += _correct
            v_total += _total

    avg_val_loss = val_loss / max(1, val_steps)

    # global val metrics
    val_iou  = v_tp / (v_tp + v_fp + v_fn + 1e-6)
    val_dice = (2*v_tp) / (2*v_tp + v_fp + v_fn + 1e-6)
    val_acc  = v_correct / max(1, v_total)

    val_prec   = (v_tp + 1e-6) / (v_tp + v_fp + 1e-6)
    val_recall = (v_tp + 1e-6) / (v_tp + v_fn + 1e-6)
    val_f1     = (2*val_prec*val_recall + 1e-6) / (val_prec + val_recall + 1e-6)

    history["val_loss"].append(avg_val_loss)
    history["val_iou"].append(val_iou)
    history["val_dice"].append(val_dice)
    history["val_acc"].append(val_acc)
    history["val_prec"].append(val_prec)
    history["val_recall"].append(val_recall)
    history["val_f1"].append(val_f1)

    print(f"Val loss: {avg_val_loss:.4f} | IoU: {val_iou:.4f} | Dice: {val_dice:.4f} | Acc: {val_acc:.4f}")
    print(f"Precision: {val_prec:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}")

# save history
hist_df = pd.DataFrame(history)
out_path = f"/content/drive/MyDrive/Data Mining/Project/Model/{history_name}"
os.makedirs(os.path.dirname(out_path), exist_ok=True)
hist_df.to_csv(out_path, index=False)
print(f"Saved training history to Drive ({history_name})")
save_path = "/content/drive/MyDrive/Data Mining/Project/Model/unet_preprocessing.pth"

torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, save_path)

print(f"Model saved to {save_path}")

Train epoch 1: 100%|██████████| 939/939 [04:46<00:00,  3.27it/s]

Epoch 1 - train avg loss: 0.8880 | IoU: 0.5583 | Dice: 0.7166 | Acc: 0.8321


Val loss: 0.7262 | IoU: 0.6416 | Dice: 0.7817 | Acc: 0.8724
Precision: 0.7308 | Recall: 0.8402 | F1: 0.7817


Train epoch 2: 100%|██████████| 939/939 [04:24<00:00,  3.55it/s]

Epoch 2 - train avg loss: 0.6557 | IoU: 0.6571 | Dice: 0.7930 | Acc: 0.8828


...

Epoch 19 - train avg loss: 0.2371 | IoU: 0.8614 | Dice: 0.9256 | Acc: 0.9581


Val loss: 0.2508 | IoU: 0.8455 | Dice: 0.9163 | Acc: 0.9553
Precision: 0.9344 | Recall: 0.8989 | F1: 0.9163


Train epoch 20: 100%|██████████| 939/939 [04:29<00:00,  3.49it/s]

Epoch 20 - train avg loss: 0.2321 | IoU: 0.8642 | Dice: 0.9271 | Acc: 0.9590


Val loss: 0.2297 | IoU: 0.8621 | Dice: 0.9259 | Acc: 0.9597
Precision: 0.9257 | Recall: 0.9262 | F1: 0.9259
Saved training history to Drive (training_unet_history_preprocessed_metrics.csv)
Model saved to /content/drive/MyDrive/Data Mining/Project/Model/unet_preprocessing.pth
